In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from config_loader import load_config
import os
import json

In [ ]:
# Load configuration
config, config_dir = load_config()
# Get the data path from the configuration
env = config['Settings']['environment']
data_path = config[env]['data_path']
demo_inferred_data_file = config['Data']['inferred_data_file']
demo_dataset_ground_truth_file = config['Data']['demo_dataset_ground_truth_file']
column_names_file = config['Data']['training_dataset_columns_file']

In [ ]:
# Read appliance names from the text file
with open(os.path.join(data_path, column_names_file), 'r') as file:
    column_names_json = json.load(file)

# Read the Parquet file
df = pd.read_parquet(os.path.join(data_path, demo_inferred_data_file))
# Read the ground truth data
df_gt = pd.read_parquet(os.path.join(data_path, demo_dataset_ground_truth_file))

print(df.head())
print(df_gt.head())

In [ ]:
appliances = column_names_json['appliances']

# Loop over each appliance
for appliance in appliances:
    df_appliance = df[df['appliance'] == appliance]
    plt.figure(figsize=(10, 5))
    plt.plot(df_appliance['timestamp'], df_appliance['value'], label=appliance)
    plt.title(f'Predictions for {appliance}')
    plt.xlabel('Timestamp')
    plt.ylabel('Predicted Value')
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()


In [ ]:
import pandas as pd
import plotly.graph_objects as go
from datetime import timedelta

# Convert timestamps to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])
df_gt['timestamp'] = pd.to_datetime(df_gt['timestamp'])

# Convert wide ground truth to long format
gt_long = df_gt.melt(
    id_vars=['timestamp'],
    value_vars=appliances,
    var_name='appliance',
    value_name='gt_value'
)

# Merge predictions and ground truth
merged = pd.merge(df, gt_long, on=['timestamp', 'appliance'], how='inner')

# Filter to first 7 days of data
start_date = merged['timestamp'].min()
end_date = start_date + timedelta(days=7)
merged_7d = merged[(merged['timestamp'] >= start_date) & (merged['timestamp'] < end_date)]

for appliance in appliances:
    df_appliance = merged_7d[merged_7d['appliance'] == appliance]

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=df_appliance['timestamp'],
        y=df_appliance['gt_value'],
        mode='lines',
        name='Ground Truth',
        line=dict(color='red')
    ))

    fig.add_trace(go.Scatter(
        x=df_appliance['timestamp'],
        y=df_appliance['value'],
        mode='lines',
        name='Prediction',
        line=dict(color='blue')
    ))

    fig.update_layout(
        title=f'Prediction vs Ground Truth: {appliance}',
        xaxis_title='Timestamp',
        yaxis_title='Power (W)',
        legend=dict(x=0, y=1.1, orientation="h"),
        height=400
    )

    fig.show()
